<a href="https://colab.research.google.com/github/genice-dev/GenIce3/blob/main/API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Installation

On Google Colaboratory, you must install GenIce3 and extra plugins by yourself.


In [ ]:
try:
    import google.colab
    %pip install git+https://github.com/genice-dev/GenIce3.git git+https://github.com/vitroid/genice3-svg.git
except:
    %pip install --no-deps ../genice3-svg
    %pip install svgwrite
    pass


## Setup

Set the working directory to the project root so that paths like `cif/MEP.cif` used in the examples resolve correctly. Logging is set to WARNING so that depolarization retry messages (Attempt N/1000 failed) do not flood the output; set to INFO in a cell if you need to debug.


In [ ]:
import os
import logging
from pathlib import Path

# Use repo root so relative paths in examples (e.g. cif/MEP.cif) resolve
REPO_ROOT = Path(".").resolve()
if (REPO_ROOT / "genice3").is_dir() and (REPO_ROOT / "examples").is_dir():
    os.chdir(REPO_ROOT)
    print("Working directory:", os.getcwd())
else:
    print("Not in repo root? Set REPO_ROOT and run os.chdir(REPO_ROOT)")

# Suppress INFO from depol loop (Attempt N/1000 failed, etc.) so notebook stays readable
logging.getLogger().setLevel(logging.WARNING)


## Basic

**Basic usage examples** of GenIce3.

- `1_reactive_properties.py`  
  - Minimal example to explore how GenIce3's *reactive properties* behave.

- `2_simple.py`  
  - Introductory example that generates a simple ice structure and exports it.


### reactive properties

`1_reactive_properties.py`


In [ ]:
from genice3.genice import GenIce3
from logging import getLogger, basicConfig, INFO

logger = getLogger("test_genice3api1")
basicConfig(level=INFO)
genice = GenIce3()
logger.info("Reactive properties:")
logger.info(f"     All: {genice.list_all_reactive_properties().keys()}")
logger.info(f"  Public: {genice.list_public_reactive_properties().keys()}")
logger.info("Settabe reactive properties:")
logger.info(f"     All: {genice.list_settable_reactive_properties().keys()}")
logger.info(f"  Public: {genice.list_public_settable_reactive_properties().keys()}")


### simple

`2_simple.py`


In [ ]:
from genice3.genice import GenIce3
from genice3.plugin import UnitCell, Exporter

# corresponding command:
# genice3 A15 --exporter gromacs
genice = GenIce3()
genice.unitcell = UnitCell("A15")
Exporter("gromacs").dump(genice)


## CIF I/O

**Reading from and writing to CIF files**.

- `cif/`  
  - Symlink to the project-root `cif/` directory. Used so examples can refer to files such as `cif/MEP.cif`.

- `4_from_cif.py`  
  - Load an ice structure from a CIF file and use it as a GenIce3 `UnitCell`.

- `5_to_cif.py`  
  - Export a structure generated by GenIce3 in CIF format.


### from cif

`4_from_cif.py`


In [ ]:
from genice3.genice import GenIce3
from genice3.plugin import UnitCell, Exporter
from logging import basicConfig, INFO

# corresponding command:
# genice3 'CIF[file=cif/MEP.cif, osite=T]' --exporter gromacs
basicConfig(level=INFO)
genice = GenIce3()
genice.unitcell = UnitCell("CIF", file="cif/MEP.cif", osite="T")
Exporter("gromacs").dump(genice)


### to cif

`5_to_cif.py`


In [ ]:
from genice3.genice import GenIce3
from genice3.plugin import UnitCell, Exporter
from logging import basicConfig, INFO

# corresponding command:
# genice3 A15 -e cif
basicConfig(level=INFO)
genice = GenIce3()
genice.unitcell = UnitCell("A15")
Exporter("cif").dump(genice)


## Doping

**Ionic substitution and group (cation group) doping**.

CLI: unitcell のイオンは **-a / --anion**, **-c / --cation**。スポット置換は **-A / --spot_anion**, **-C / --spot_cation**。

- `3_doped.py`  
  - Basic usage of anion/cation substitutional doping (unitcell + spot).

- `9_ion_group.py`  
  - Attach *groups* to cations inside a unit cell.

- `11_ion_group_unitcell.py`  
  - Use `cation_groups` defined in the unit cell and expand them to the replicated supercell.


### ion group unitcell

`11_ion_group_unitcell.py`


In [ ]:
from logging import basicConfig, INFO
import numpy as np
from genice3.genice import GenIce3
from genice3.plugin import UnitCell, Exporter

# corresponding command:
# genice3 A15 --cation 0=N :group 1=methyl 6=methyl 3=methyl 4=methyl \
#   --anion 2=Cl --rep 2 2 2 --exporter gromacs :water_model 4site

basicConfig(level=INFO)

genice = GenIce3(
    replication_matrix=np.array([[2, 0, 0], [0, 2, 0], [0, 0, 2]]),
    seed=43,
)

# 単位胞内の anion/cation と cation_groups（カチオンの腕の group 指定）
genice.unitcell = UnitCell(
    "A15",
    anion={2: "Cl"},
    cation={0: "N"},
    cation_groups={0: {1: "methyl", 6: "methyl", 3: "methyl", 4: "methyl"}},
)

genice.depol_loop = 5000  # notebook: more attempts
Exporter("gromacs").dump(
    genice,
    water_model="4site",
)


### doped

`3_doped.py`


In [ ]:
from logging import basicConfig, DEBUG, INFO
import numpy as np
from genice3.genice import GenIce3
from genice3.plugin import UnitCell, Exporter, Molecule

# corresponding command:
# genice3 "A15[shift=(0.1,0.1,0.1), anion.0=Cl, cation.6=Na, density=0.8]" \
#   --rep 2 2 2 \
#   --spot_anion 1=Cl --spot_anion 35=Br \
#   --spot_cation 1=Na --spot_cation 35=K \
#   --exporter gromacs :water_model 4site \
#   --seed 42 --depol_loop 2000 -D

basicConfig(level=INFO)

# GenIce3インスタンスを作成
# seed: 乱数シード
# depol_loop: 分極ループの反復回数
# replication_matrix: 複製行列（2x2x2の対角行列）
# spot_anions: 特定の水分子をアニオンで置換（水分子インデックス: イオン名）。CLI は -A / --spot_anion
# spot_cations: 特定の水分子をカチオンで置換（水分子インデックス: イオン名）。CLI は -C / --spot_cation
# 注意: debugはGenIce3のコンストラクタでは受け付けられない（ログレベルの設定はbasicConfigで行う）
genice = GenIce3(
    seed=42,
    depol_loop=2000,
    replication_matrix=np.array([[2, 0, 0], [0, 2, 0], [0, 0, 2]]),
    spot_anions={
        1: "Cl",
    },
    spot_cations={
        51: "Na",
    },
)

# 単位セルを設定
# shift: シフト（分数座標）
# anion: 単位胞内の格子サイトをアニオンで置換（サイトインデックス: イオン名）。CLI は -a / --anion
# cation: 単位胞内の格子サイトをカチオンで置換（サイトインデックス: イオン名）。CLI は -c / --cation
# density: 密度（g/cm³）
# ケージ情報が必要な場合は Exporter("cage_survey").dump(genice, file) でJSON出力可能
genice.unitcell = UnitCell(
    "A15",
    shift=(0.1, 0.1, 0.1),
    anion={15: "Cl"},
    cation={21: "Na"},
    density=0.8,
)


# エクスポーターで出力
Exporter("gromacs").dump(
    genice,
    water_model="4site",
)


### ion group

`9_ion_group.py`


In [ ]:
from logging import basicConfig, DEBUG, INFO
import numpy as np
from genice3.genice import GenIce3
from genice3.plugin import UnitCell, Exporter, Molecule

# corresponding command:
# genice3 "A15[shift=(0.1,0.1,0.1), anion.0=Cl, cation.6=Na, density=0.8]" \
#   --rep 2 2 2 \
#   --spot_anion 1=Cl --spot_anion 35=Br \
#   --spot_cation 1=Na --spot_cation 35=K \
#   --exporter gromacs :water_model 4site \
#   --seed 42 --depol_loop 2000 -D

basicConfig(level=INFO)

# GenIce3インスタンスを作成
# seed: 乱数シード
# depol_loop: 分極ループの反復回数
# replication_matrix: 複製行列（2x2x2の対角行列）
# spot_anions / spot_cations: 水分子インデックス -> イオン名。CLI は -A / --spot_anion, -C / --spot_cation
# spot_cation_groups: group サブオプション（サイト -> {ケージID -> group名}）。
# YAML/CLI のネスト形式で使う "ion" キーは Python API では不要（別引数で渡す）。
genice = GenIce3(
    seed=42,
    depol_loop=2000,
    replication_matrix=np.array([[2, 0, 0], [0, 2, 0], [0, 0, 2]]),
    spot_anions={1: "Cl"},
    spot_cations={51: "N"},
    spot_cation_groups={
        51: {12: "methyl", 48: "methyl", 49: "methyl", 50: "methyl"},
    },
)

# 単位セルを設定
# anion / cation: 単位胞内の格子サイトをイオンで置換（サイトインデックス: イオン名）。CLI は -a / --anion, -c / --cation
# density: 密度（g/cm³）
# ケージ情報が必要な場合は Exporter("cage_survey").dump(genice, file) でJSON出力可能
genice.unitcell = UnitCell(
    "A15",
    shift=(0.1, 0.1, 0.1),
    density=0.8,
)


# エクスポーターで出力
Exporter("yaplot").dump(
    genice,
)


## Guest occupancy

**Guest occupancy and cage-related operations** in clathrate hydrates.

- `6_with_guests.py`  
  - Place guest molecules with specified occupancy for each cage type.

- `8_cage_survey.py`  
  - Survey cage positions and types in the structure.


### with guests

`6_with_guests.py`


In [ ]:
from logging import basicConfig, DEBUG, INFO
import numpy as np
from genice3.genice import GenIce3
from genice3.plugin import UnitCell, Exporter, Molecule
from genice3.cli.options import parse_guest_option, parse_spot_guest_option

# corresponding command (guest/spot_guest は基底オプション、exporterの外で指定):
# genice3 "A15[shift=(0.1,0.1,0.1), anion.0=Cl, cation.6=Na, density=0.8]" \
#   --rep 2 2 2 \
#   --guest A12=me --guest A14=et --spot_guest 0=4site \
#   --exporter gromacs :water_model 4site \
#   --seed 42 --depol_loop 2000 -D

basicConfig(level=INFO)

# GenIce3インスタンスを作成
# guests: ケージタイプごとのゲスト分子指定（parse_guest_optionで raw dict を変換）
# spot_guests: 特定ケージへのゲスト分子指定（parse_spot_guest_optionで raw dict を変換）
genice = GenIce3(
    seed=42,
    depol_loop=2000,
    replication_matrix=np.array([[2, 0, 0], [0, 2, 0], [0, 0, 2]]),
    guests=parse_guest_option({"A12": "me", "A14": "et"}),
    spot_guests=parse_spot_guest_option({0: "4site"}),
)

# 単位セルを設定
# shift: シフト（分数座標）
# anion / cation: 単位胞内の格子サイトをイオンで置換（サイトインデックス: イオン名）。CLI は -a / --anion, -c / --cation
# density: 密度（g/cm³）
# ケージ情報が必要な場合は Exporter("cage_survey").dump(genice, file) でJSON出力可能
genice.unitcell = UnitCell(
    "A15",
    shift=(0.1, 0.1, 0.1),
    density=0.8,
)


# エクスポーターで出力（guest/spot_guest は GenIce3 に設定済み）
Exporter("gromacs").dump(
    genice,
    water_model="4site",
)


### cage survey

`8_cage_survey.py`


In [ ]:
"""DOH 構造で cage_survey を使う例（Python API）"""
from genice3.genice import GenIce3
from genice3.plugin import UnitCell, Exporter
from logging import basicConfig, INFO

# 対応するコマンド:
#   genice3 DOH -e cage_survey
basicConfig(level=INFO)
genice = GenIce3()
genice.unitcell = UnitCell("DOH")
Exporter("cage_survey").dump(genice)


## Polarization

**Polarization and dipole optimization**.

- `7_polarized.py`  
  - Generate a structure with a specified polarization using `target_pol` and `depol_loop`.


### polarized

`7_polarized.py`


In [ ]:
# 分極した氷の作り方 (1) コンストラクタで target_pol を指定
# corresponding command: 7_polarized_1.sh または 7_polarized_1_flat.sh

from logging import basicConfig, INFO
import numpy as np
from genice3.genice import GenIce3
from genice3.plugin import UnitCell, Exporter

basicConfig(level=INFO)

genice = GenIce3(
    seed=114,
    depol_loop=1000,
    replication_matrix=np.diag([2, 2, 2]),
    target_pol=np.array([4.0, 0.0, 0.0]),
)
genice.unitcell = UnitCell("1h")

Exporter("_pol").dump(genice)


## Unit cell transform

**Extending or transforming the unit cell**.

- `10_extend_unitcell.py`  
  - Use `replication_matrix` to extend the unit cell and build a larger supercell.


### extend unitcell

`10_extend_unitcell.py`


In [ ]:
#!/usr/bin/env python3
"""
拡大胞を新たに単位胞とする unitcell プラグインを出力する例。

対応する CLI: examples/api/10_extend_unitcell.sh
対応する YAML: examples/api/10_extend_unitcell.yaml

生成された A15e.py は、rep=1 1 1 で同じ構造を再現する unitcell プラグインです。
"""

from pathlib import Path

from genice3.genice import GenIce3
from genice3.plugin import safe_import

# A15 単位胞、複製行列で拡大
unitcell = safe_import("unitcell", "A15").UnitCell()
genice = GenIce3(unitcell=unitcell)
genice.replication_matrix = [[1, 1, 0], [-1, 1, 0], [0, 0, 1]]

# python exporter で unitcell プラグインのソースを取得
exporter = safe_import("exporter", "python")
exporter.dump(genice)


## Topological defects

**Topological defects** (Bjerrum defects, H3O⁺, OH⁻).

- `12_topological_defect.py`  
  - Introduce hydronium (H3O⁺) and hydroxide (OH⁻) defects at specified coordinates.

- `13_topological_defect2.py`  
  - Introduce Bjerrum L and D defects at specified coordinates.

Additional implementations for the same topics (e.g., CLI- or config-file–driven variants) may be added here in the future.


### topological defect

`12_topological_defect.py`


In [ ]:
"""
トポロジカル欠陥（Hydronium/Hydroxide）を座標指定で埋め込むサンプル。
"""

from __future__ import annotations

from logging import basicConfig, INFO

import numpy as np

from genice3.genice import GenIce3
from genice3.plugin import UnitCell, Exporter
from genice3.util import find_nearest_sites_pbc

# -----------------------------------------------------------------------------
# サンプル本体
# -----------------------------------------------------------------------------

basicConfig(level=INFO)

genice = GenIce3(
    replication_matrix=np.array([[2, 0, 0], [0, 2, 0], [0, 0, 2]]),
)
genice.unitcell = UnitCell("A15")

# 欠陥を置きたい位置を分数座標で指定（各 2 点ずつ）セル座標に変換して。
celli = np.linalg.inv(genice.cell)
H3O_positions = np.array([[0.0, 0.0, 0.0], [1.0, 1.0, 1.0]]) @ celli
OH_positions = np.array([[1.0, 0.0, 0.0], [2.0, 1.0, 1.0]]) @ celli
H3O_sites = find_nearest_sites_pbc(H3O_positions, genice.lattice_sites, genice.cell)
OH_sites = find_nearest_sites_pbc(OH_positions, genice.lattice_sites, genice.cell)

genice.add_spot_hydronium(H3O_sites)
genice.add_spot_hydroxide(OH_sites)

Exporter("gromacs").dump(
    genice,
    water_model="3site",
)


### topological defect2

`13_topological_defect2.py`


In [ ]:
"""
トポロジカル欠陥（Bjerrum）を座標指定で埋め込むサンプル。
"""

from __future__ import annotations

from logging import basicConfig, INFO

import numpy as np

from genice3.genice import GenIce3
from genice3.plugin import UnitCell, Exporter
from genice3.util import find_nearest_edges_pbc

# -----------------------------------------------------------------------------
# サンプル本体
# -----------------------------------------------------------------------------

basicConfig(level=INFO)

genice = GenIce3(
    replication_matrix=np.array([[2, 0, 0], [0, 2, 0], [0, 0, 2]]),
)
genice.unitcell = UnitCell("A15")

# 欠陥を置きたい位置を分数座標で指定（各 2 点ずつ）セル座標に変換して。
celli = np.linalg.inv(genice.cell)
D_positions = np.array([[0.0, 0.0, 0.0], [1.0, 1.0, 1.0]]) @ celli
L_positions = np.array([[1.0, 0.0, 0.0], [2.0, 1.0, 1.0]]) @ celli
D_edges = find_nearest_edges_pbc(
    D_positions, genice.graph, genice.lattice_sites, genice.cell
)
L_edges = find_nearest_edges_pbc(
    L_positions, genice.graph, genice.lattice_sites, genice.cell
)

genice.add_bjerrum_D(D_edges)
genice.add_bjerrum_L(L_edges)

Exporter("gromacs").dump(
    genice,
    water_model="3site",
)
